In [ ]:
import matplotlib

import numpy as np

import pandas as pd
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/breaset cancer/data.csv')
col=list(data.columns.values)
col

['id',
 'diagnosis',
 'radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst',
 'Unnamed: 32']

In [ ]:
dataF=data

In [ ]:

ddf = dataF.drop(dataF.columns[[0,1,31]], axis=1)
ddf.shape

(569, 30)

In [ ]:
y=data["diagnosis"]
x=ddf

In [ ]:
ddf[:2]

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,Unnamed: 32
0,370,1,455,452,444,531,521,526,416,483,525,169,520,515,277,489,460,400,439,489,425,41,497,516,358,516,520,483,485,0
1,426,186,486,513,120,215,321,396,219,91,424,91,396,450,145,135,177,336,81,309,419,199,465,511,158,210,278,401,222,374


In [ ]:
targetclass=data['diagnosis']
from sklearn import preprocessing
import pandas as pd

le = preprocessing.LabelEncoder()
targetclass1 = le.fit_transform(targetclass)

In [ ]:
ddf=ddf.apply(le.fit_transform)

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(ddf, targetclass1.ravel())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



In [ ]:
from collections import Counter
Counter(y_train_res)

Counter({0: 357, 1: 357})

In [ ]:
>>> from collections import Counter
>>> from sklearn.datasets import make_classification
>>> from imblearn.over_sampling import ADASYN # doctest: +NORMALIZE_WHITESPACE
ada = ADASYN(ratio='all')
>>> X_res, y_res = ada.fit_resample(ddf, targetclass1)
>>> print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({1: 363, 0: 357})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res,test_size=0.25, random_state=200,shuffle=True)

In [ ]:
# develop an mlp for blobs dataset
from sklearn.datasets.samples_generator import make_blobs
from keras.utils import to_categorical
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

from keras.layers import Dense
from matplotlib import pyplot
# generate 2d classification dataset
trainy = to_categorical(y_train)
testy = to_categorical(y_test)

# split into train and test
n_train = 100
trainX=X_train
testX=X_test 

print(trainX.shape, testX.shape)
# define model
model = Sequential()
model.add(Dense(128, input_dim=30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=500, verbose=0)
# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=2)
_, test_acc = model.evaluate(testX, testy, verbose=2)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# learning curves of model accuracy

(535, 30) (179, 30)
17/17 - 0s - loss: 0.0242 - accuracy: 0.9888
6/6 - 0s - loss: 0.1412 - accuracy: 0.9553
Train: 0.989, Test: 0.955


In [ ]:

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=50) 
rfc.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = rfc.predict(X_test)
accuracy_score(y_test, y_pred_class)

0.9832402234636871

In [ ]:

from sklearn.svm import SVC

svm = SVC(kernel='linear', C=2.0, random_state=5)
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
accuracy_score(y_test,y_pred)
#print(classification_report(y_test,y_pred))

0.9553072625698324

In [ ]:

from sklearn.ensemble import GradientBoostingClassifier
a = GradientBoostingClassifier(max_depth=15, learning_rate=0.2, n_estimators=300, random_state=52)

a_pred = a.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,a_pred)
#print(classification_report(y_test,a_pred))

0.9553072625698324

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 2. instantiate a logistic regression model
logreg = LogisticRegression(random_state=1000, solver='liblinear',multi_class='ovr',C=3.0)
# 3. train the model using X_train_dtm
x=logreg.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)
#print(classification_report(y_test, y_pred_class))

0.9720670391061452

In [ ]:
>>> from sklearn.neighbors import KNeighborsClassifier
>>> neigh = KNeighborsClassifier(n_neighbors=3)
x=neigh.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = neigh.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)

0.994413407821229

In [ ]:
print("start PCA")

start PCA


In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
#union = FeatureUnion([("pca", PCA(n_components=3)),("svd", TruncatedSVD(n_components=2))])
union =FeatureUnion([("pca", PCA(n_components=15))])
fpca=union.fit_transform(X_train_res)  
print(fpca.shape)

(714, 15)


In [ ]:
x1=y_train_res

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fpca,x1,test_size=0.25, random_state=200,shuffle=True)

In [ ]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))
print(Counter(x1))
print(len(x1))
print(X_train.shape)
print(len(y_train))
print(len(y_test))

Counter({1: 271, 0: 264})
Counter({0: 93, 1: 86})
Counter({1: 357, 0: 357})
714
(535, 15)
535
179


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
print("Radndomforest classifer after PCA");
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=50) 
rfc.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = rfc.predict(X_test)
accuracy_score(y_test, y_pred_class)

Radndomforest classifer after PCA


0.9832402234636871

In [ ]:
print("PCA K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, fpca,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

PCA K fold validation
Accuracy: 0.97 (+/- 0.04)


In [ ]:
print("SVC classifer after PCA")
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=2.0, random_state=5)
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
accuracy_score(y_test,y_pred)
#print(classification_report(y_test,y_pred))

SVC classifer after PCA


0.9776536312849162

In [ ]:
print("PCA K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, fpca,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

PCA K fold validation
Accuracy: 0.98 (+/- 0.02)


In [ ]:
print("GBM classifier after PCA");
from sklearn.ensemble import GradientBoostingClassifier
a = GradientBoostingClassifier(max_depth=15, learning_rate=0.2, n_estimators=300, random_state=52)

a_pred = a.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,a_pred)
#print(classification_report(y_test,a_pred))

GBM classifier after PCA


0.9608938547486033

In [ ]:
print("PCA K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(a, fpca,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

PCA K fold validation
Accuracy: 0.93 (+/- 0.03)


In [ ]:
print("LR classifer after PCA")
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 2. instantiate a logistic regression model
logreg = LogisticRegression(random_state=1000, solver='liblinear',multi_class='ovr',C=3.0)
# 3. train the model using X_train_dtm
x=logreg.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)
#print(classification_report(y_test, y_pred_class))

LR classifer after PCA


0.9720670391061452

In [ ]:
print("PCA K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logreg, fpca,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

PCA K fold validation
Accuracy: 0.98 (+/- 0.02)


In [ ]:
>>> from sklearn.neighbors import KNeighborsClassifier
>>> neigh = KNeighborsClassifier(n_neighbors=3)
x=neigh.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = neigh.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)

1.0

In [ ]:
print("PCA K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(neigh, fpca,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

PCA K fold validation
Accuracy: 0.97 (+/- 0.02)


In [ ]:
print("svm+gbm+RF after PCA")
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
lr=LogisticRegression()
from sklearn.linear_model import SGDClassifier

rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=100) 
svm = SVC(kernel='linear', C=2.0, random_state=500,probability=True)
gbm= GradientBoostingClassifier(max_depth=100, learning_rate=0.1, n_estimators=50, random_state=52)
#svm=SVC(probability=True)
#vc = VotingClassifier(estimators=[('nb',nb),('clf',clf)],voting='hard')
vc = VotingClassifier(estimators=[('rfc', rfc),('svm',svm),('gbm',gbm)],voting='hard')
vc.fit(X_train,y_train)
y_pre = vc.predict(X_test)
rst=accuracy_score(y_test, y_pre)
#print(classification_report(y_test, y_pre))
rst

svm+gbm+RF after PCA


0.916083916083916

In [ ]:
print("PCA K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(vc, fpca,data.diagnosis, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

PCA K fold validation
Accuracy: 0.95 (+/- 0.03)


In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
#union = FeatureUnion([("pca", PCA(n_components=3)),("svd", TruncatedSVD(n_components=2))])
union =FeatureUnion([("svd", TruncatedSVD(n_components=15))])
fsvd=union.fit_transform(X_train_res)  
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fsvd,x1,test_size=0.25, random_state=200,shuffle=True)

In [ ]:
print("Radndomforest classifer after SVD");
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=100) 
rfc.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = rfc.predict(X_test)
accuracy_score(y_test, y_pred_class)

Radndomforest classifer after SVD


0.9720670391061452

In [ ]:
print("SVD K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, fsvd,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

SVD K fold validation
Accuracy: 0.96 (+/- 0.02)


In [ ]:
print("SVC classifer after SVD")
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=2.0, random_state=500)
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
accuracy_score(y_test,y_pred)
#print(classification_report(y_test,y_pred))

SVC classifer after SVD


0.9720670391061452

In [ ]:
print("SVD K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, fsvd,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

SVD K fold validation
Accuracy: 0.97 (+/- 0.02)


In [ ]:
print("GBM classifier after SVD");
from sklearn.ensemble import GradientBoostingClassifier
a = GradientBoostingClassifier(max_depth=100, learning_rate=0.1, n_estimators=50, random_state=52)

a_pred = a.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,a_pred)
#print(classification_report(y_test,a_pred))

GBM classifier after SVD


0.9385474860335196

In [ ]:
print("SVD K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(a, fsvd,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

SVD K fold validation
Accuracy: 0.94 (+/- 0.03)


In [ ]:
print("LR classifer after SVD")
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 2. instantiate a logistic regression model
logreg = LogisticRegression(random_state=1000, solver='liblinear',multi_class='ovr',C=3.0)
# 3. train the model using X_train_dtm
x=logreg.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)
#print(classification_report(y_test, y_pred_class))

LR classifer after SVD


0.9888268156424581

In [ ]:
print("SVD K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logreg, fsvd,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

SVD K fold validation
Accuracy: 0.96 (+/- 0.02)


In [ ]:
>>> from sklearn.neighbors import KNeighborsClassifier
>>> neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = neigh.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)

0.9888268156424581

In [ ]:
print("SVD K fold validation")
from sklearn import svm
from sklearn.model_selection import cross_val_score
scores = cross_val_score(neigh, fsvd,x1, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

SVD K fold validation
Accuracy: 0.96 (+/- 0.03)


In [ ]:
print("SVD end")

SVD end


In [ ]:
print("chi square starts")

chi square starts


In [ ]:
from sklearn.datasets import load_iris 
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 
 
chi2_features = SelectKBest(chi2,k=15) 
X_kbest= chi2_features.fit_transform(X_train_res,x1) 
print("for chi2 k cross validation")

for chi2 k cross validation


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_kbest,x1,test_size=0.25, random_state=200,shuffle=True)

In [ ]:
X_train.shape

(535, 15)

In [ ]:
print("SVC classifer after SVD")
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=2.0, random_state=500)
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
accuracy_score(y_test,y_pred)
#print(classification_report(y_test,y_pred))

SVC classifer after SVD


0.9664804469273743

In [ ]:
print("LR classifer");print("before")
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 2. instantiate a logistic regression model
logreg = LogisticRegression(random_state=1000, solver='liblinear',multi_class='ovr',C=3.0)
# 3. train the model using X_train_dtm
x=logreg.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test)
# calculate accuracy
accuracy_score(y_test, y_pred_class)
#print(classification_report(y_test, y_pred_class))

LR classifer
before


0.9497206703910615

In [ ]:
print("Radndomforest classifer");print("before")
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=10) 
rfc.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = rfc.predict(X_test)
accuracy_score(y_test, y_pred_class)
#print(classification_report(y_test, y_pred_class))

Radndomforest classifer
before


0.9664804469273743

In [ ]:
print("GBM classifier after SVD");
from sklearn.ensemble import GradientBoostingClassifier
a = GradientBoostingClassifier(max_depth=100, learning_rate=0.1, n_estimators=50, random_state=52)

a_pred = a.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,a_pred)
#print(classification_report(y_test,a_pred))

GBM classifier after SVD


0.9553072625698324

In [ ]:
>>> from sklearn.neighbors import KNeighborsClassifier
>>> neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = neigh.predict(X_test)
accuracy_score(y_test, y_pred_class)
#print(classification_report(y_test, y_pred_class))

0.9720670391061452

In [ ]:
print("VOting")


VOting


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
lr=LogisticRegression()
from sklearn.linear_model import SGDClassifier
logreg = LogisticRegression(random_state=1000, solver='liblinear',multi_class='ovr',C=3.0)
rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=100) 
svm = SVC(kernel='linear', C=2.0, random_state=500,probability=True)
gbm= GradientBoostingClassifier(max_depth=100, learning_rate=0.1, n_estimators=50, random_state=52)
#svm=SVC(probability=True)
#vc = VotingClassifier(estimators=[('nb',nb),('clf',clf)],voting='hard')
vc = VotingClassifier(estimators=[('rfc', rfc),('svm',svm),('gbm',gbm),('logreg',logreg)],voting='soft')
vc.fit(X_train,y_train)
y_pre = vc.predict(X_test)
rst=accuracy_score(y_test, y_pre)
#print(classification_report(y_test, y_pre))
rst

0.9370629370629371

In [ ]:
print("Chi2 K fold validation")
from sklearn.model_selection import cross_val_score
scores = cross_val_score(vc, X_kbest,data.diagnosis, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Chi2 K fold validation
Accuracy: 0.96 (+/- 0.04)


In [ ]:
print("After chi-square")

After chi-square


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
lr=LogisticRegression()
from sklearn.linear_model import SGDClassifier
rfc = RandomForestClassifier(n_estimators=300, random_state=2,max_depth=100) 
svm = SVC(kernel='linear', C=2.0, random_state=500,probability=True)
gbm= GradientBoostingClassifier(max_depth=100, learning_rate=0.1, n_estimators=50, random_state=52)

#svm=SVC(probability=True)
#vc = VotingClassifier(estimators=[('nb',nb),('clf',clf)],voting='hard')
vc = VotingClassifier(estimators=[('rfc', rfc),('svm',svm),('gbm',gbm)],voting='hard')
vc.fit(X_kbest_features,y_train)
y_pre = vc.predict(X_kbest_features1)
rst=accuracy_score(y_test, y_pre)
#print(classification_report(y_test, y_pre))
rst

0.9440559440559441

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
logreg_x=['RF',
'SVM',
'GBM',
'LR',
'RSG'

]
a=[0.95,
0.92,
0.91,
0.91,
0.92,

]
b=[0.90,
0.92,
0.88,
0.92,
0.92,

]
c=[0.707,
0.920,
0.796,
0.876,
]

fig, ax = plt.subplots(figsize=(7, 4),dpi=400)
# only these two lines are calibration curves
plt.plot(logreg_x,a, marker='o', linewidth=1, label='Under-Sampling')
plt.plot(logreg_x, b, marker='o', linewidth=1, label='Over-Sampling')
plt.plot(logreg_x, c, marker='o', linewidth=1, label='Without Resampling')

# reference line, legends, and axis labels
line = mlines.Line2D([0, 1], [0, 1], color='black')
transform = ax.transAxes
line.set_transform(transform)
#ax.add_line(line)
#fig.suptitle('Comparison')
ax.set_xlabel('Machine Learning Models')
ax.set_ylabel('Score')
plt.legend()
plt.savefig('overall.png')
plt.show()